In [1]:
!pip install gradio

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 47.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.0/140.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 37.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determin

In [29]:
import cv2
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

import gradio as gr

# 手绘:
img_cartoon_art = pipeline(Tasks.image_portrait_stylization, 
                       'damo/cv_unet_person-image-cartoon-handdrawn_compound-models')
# 3d卡通
img_cartoon_3d = pipeline(Tasks.image_portrait_stylization, 
                       model='damo/cv_unet_person-image-cartoon-3d_compound-models', model_revision='v1.0.0')

2023-08-15 22:20:22,274 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
Downloading: 100%|██████████| 22.6k/22.6k [00:00<00:00, 4.23MB/s]
Downloading: 100%|██████████| 5.62M/5.62M [00:00<00:00, 125MB/s]
Downloading: 100%|██████████| 5.62M/5.62M [00:00<00:00, 51.0MB/s]
Downloading: 100%|██████████| 499/499 [00:00<00:00, 418kB/s]
Downloading: 100%|██████████| 3.28M/3.28M [00:00<00:00, 64.2MB/s]
Downloading: 100%|██████████| 4.27M/4.27M [00:00<00:00, 41.5MB/s]
Downloading: 100%|██████████| 3.95M/3.95M [00:00<00:00, 107MB/s]
Downloading: 100%|██████████| 29.9M/29.9M [00:00<00:00, 91.6MB/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 7.20MB/s]
Downloading: 100%|██████████| 448k/448k [00:00<00:00, 16.5MB/s]
Downloading: 100%|██████████| 7.02k/7.02k [00:00<00:00, 4.72MB/s]
Downloading: 100%|██████████| 116M/116M [00:00<00:00, 335MB/s] 
2023-08-15 22:20:32,202 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/damo/cv_unet

In [31]:
def art_style(image_input):
    result = img_cartoon_art(image_input)
    image_output = result[OutputKeys.OUTPUT_IMG]/255 - 0.5
    return image_output

def ThreeD_style(image_input):
    result = img_cartoon_3d(image_input)
    image_output = result[OutputKeys.OUTPUT_IMG]/255 - 0.5
    return image_output


In [23]:

# img_path = '/mnt/workspace/me.jpg'
# result = img_cartoon_3d(img_path)

# cv2.imwrite('美体抠图3d_result.png', result[OutputKeys.OUTPUT_IMG])
# result[OutputKeys.OUTPUT_IMG]/255 - 0.5

/opt/conda/lib/python3.8/site-packages/modelscope/models/cv/cartoon/mtcnn_pytorch/src/matlab_cp2tform.py:86: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


In [32]:
with gr.Blocks(title='肖像生成卡通APP') as demo:
    gr.Markdown(
        """
        ## 由肖像生成卡通肖像
        
            + 手绘卡通
            + 3D卡通   
    
        """)
    with gr.Row():
        image_input = gr.Image()
        with gr.Tab("handdrawn_Style"):
            image_output1 = gr.Image(shape=(20,30))
            image_button1 = gr.Button("一键生成手绘卡通")
        with gr.Tab("3D Style"):
            image_output2 = gr.Image(shape=(20,30))
            image_button2 = gr.Button("一键生成3D卡通")

    image_button1.click(art_style, inputs=image_input, outputs=image_output1)
    image_button2.click(ThreeD_style, inputs=image_input, outputs=image_output2)

demo.launch( share=False,show_error=True)

SyntaxError: unmatched ')' (576431176.py, line 16)